In [1]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon
import matplotlib.pyplot as plt
import itertools
import statsmodels.api as sm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
sectors = pd.read_excel('49_Industry_Portfolios_Daily.xlsx', sheet_name='LSSector')
factors = pd.read_excel('49_Industry_Portfolios_Daily.xlsx', sheet_name='Factors')

In [3]:
stock_data = pd.read_csv('StockData.csv', header=None)

In [4]:
stock_dates = pd.read_csv('StockDates.csv', header=None)

In [5]:
print("Sector HEAD \n")
print(sectors.head())
print("Sector shape \n")
print(sectors.shape)
print("Sector describe \n")
print(sectors.describe)

print("Factor HEAD \n")
print(factors.head())
print("factors shape \n")
print(factors.shape)
print("factor describe \n")
print(factors.describe)

print("Sector HEAD \n")
print(stock_dates.head())
print("Sector HEAD \n")
print(stock_dates.shape)
print("Sector HEAD \n")
print(stock_dates.describe)

print(stock_data.head())
print(stock_data.shape)
print(stock_data.describe)

Sector HEAD 

        Row  Market_RF     Agric      Food      Soda      Beer     Smoke  \
0  19900102       1.44  0.384446 -1.528202 -1.245530 -0.458392  1.087373   
1  19900103      -0.06 -0.816019 -0.637575 -0.436020 -0.537150  1.133026   
2  19900104      -0.71  1.233780 -0.164637  0.783768 -0.039612 -2.487524   
3  19900105      -0.85  0.613737  0.021689  1.553723 -0.112296  1.200203   
4  19900108       0.30 -0.175907 -0.388125  0.404098  0.949752 -2.911131   

       Toys       Fun     Books  ...     Boxes     Trans     Whlsl     Rtail  \
0  0.228338  0.425476  0.001780  ...  0.260234  0.770604 -0.102255  1.429006   
1 -0.796181  0.353105 -0.927158  ...  0.383740 -0.677109 -0.456573  0.650458   
2  0.071861  0.633411  0.730303  ...  0.320926  0.300883 -0.062777  0.552087   
3  0.681439  0.771323 -0.202398  ...  0.500320  0.098296  0.375887  0.293823   
4 -0.505096 -0.251526 -0.210212  ...  0.585299 -0.080457 -0.353136 -0.128290   

      Meals     Banks     Insur     RlEst       

In [6]:
sum_column = stock_dates[stock_dates.columns[0]].astype(str).copy()
for column in stock_dates.columns[1:]:
    sum_column += stock_dates[column].astype(str) if stock_dates[column].astype(str)[0] != '.' else ''

In [7]:
sum_column = pd.to_datetime(sum_column, format='%d%m%Y')

In [8]:
stock_data['date'] = sum_column
del sum_column
del stock_dates
stock_data['date'].head()

0   1990-01-03
1   1990-01-04
2   1990-01-05
3   1990-01-06
4   1990-01-09
Name: date, dtype: datetime64[ns]

In [9]:
factors['Unnamed: 0'] = pd.to_datetime(factors['Unnamed: 0'], format='%Y%m%d')

In [10]:
sectors['Row'] = pd.to_datetime(sectors['Row'], format='%Y%m%d')


In [11]:
def get_null_rows(y):
    df_mask = pd.notna(y)
    return df_mask

def _prepare_X_y(stock_data, predictor_frame, index, predictor_on='Row'):
    stock_1_data = stock_data[['date', index]]

    stock_1_sectors = pd.merge(predictor_frame, stock_1_data,left_on=predictor_on, right_on='date')
    columns = list(stock_1_sectors.columns).copy()
    columns[-1] = 'stock_price'
    stock_1_sectors.columns = columns
    stock_1_sectors = stock_1_sectors.sort_values(by=[predictor_on])
    if 'Market_RF' in stock_1_sectors.columns:
        stock_1_sectors.drop(labels=['Market_RF'], axis=1, inplace=True)
    elif 'Mkt-RF' in stock_1_sectors.columns:
        stock_1_sectors.drop(labels=['Mkt-RF', 'RF'], axis=1, inplace=True)
    stock_1_sectors.drop(labels=['date'], axis=1, inplace=True)
    daily_stock_returns = stock_1_sectors['stock_price'].copy().pct_change()


    X = stock_1_sectors[stock_1_sectors.columns.difference(['stock_price', predictor_on])]
    dates = stock_1_sectors[predictor_on].copy()
    y = daily_stock_returns
    null_mask = get_null_rows(y)
    y = y.dropna()
    X = X[null_mask]
    dates = dates[null_mask]
    return X, y, dates

def get_alpha_beta_mse(X, y, dates, training_length):
    alphas = []
    betas = []
    mse = []
    r2_scores = []
    p_values = []
    for row_break in range(0, y.shape[0], training_length):
        if row_break + training_length > y.shape[0]:
            break
        X_train, y_train = X.iloc[row_break: row_break + training_length].copy(), \
                           y.iloc[row_break: row_break + training_length].copy()

        X_train = sm.add_constant(X_train)

        if 'const' not in X_train.columns:
            continue

        regression_model = sm.OLS(y_train * 100, X_train)
        regression_results = regression_model.fit()
        alphas.append(round(float(regression_results.params.const), 5))
        betas.append([round(float(regression_results.params.tolist()[i]), 5)
                      for i, param in enumerate(regression_results.params.index) if param != 'const'])
        # betas.append([round(float(coef), 5) for coef in regression_model.coef_])
        r2_scores.append(regression_results.rsquared)
        mse.append(regression_results.mse_total)
        p_values.append(regression_results.pvalues)

    betas = pd.DataFrame(betas, columns=X.columns)
    p_values = pd.concat(p_values, axis=1).T
    return alphas, betas, mse, r2_scores, p_values

def perform_significance_test(alpha_1=None, alpha_2=None, betas_1=None, betas_2=None):
    significance = dict()
    if alpha_1 is not None and alpha_2 is not None:
        significance['alpha'] = dict()
        significance['alpha']['less'] = wilcoxon(alpha_1, alpha_2, alternative='less')
        significance['alpha']['two-sided'] = wilcoxon(alpha_1, alpha_2, alternative='two-sided')
    elif betas_2 is not None and betas_1 is not None:
        significance['betas'] = dict()
        significance['betas']['two-sided'] = wilcoxon(betas_1, betas_2, alternative='two-sided')
        significance['betas']['less'] = wilcoxon(betas_1, betas_2, alternative='less')
    else:
        raise ValueError('missing arguments to function')
    return significance

def run_experiment_single_training_length(regressor, regressor_on, training_length, stocks):
    results = list()
    # remove stocks with more than 40% missing values
    columns_keep = stocks.columns[stocks.isnull().mean() < 0.4]
    print(len(columns_keep))

    for i, column in enumerate(columns_keep[:300]):
        if column == 'date':
            continue
        results.append(dict())
        if i % 100 == 0:
            print("Running on", column)
        X, y, dates = _prepare_X_y(stocks, regressor, column, predictor_on=regressor_on)
        if any(y.isna()):
            counts = y.isna().value_counts(normalize=True)
            if False in counts and counts[False] < 0.75:
                print('skipping stock', column)
                continue
            elif False not in counts:
                print('skipping stock', column)
                continue
        alphas, betas, mse, r2_scores, p_values = get_alpha_beta_mse(X, y, dates, training_length)
        # print(betas)
        results[i]['column'] = column
        results[i]['alpha'] = np.mean(alphas)
        results[i]['betas'] = betas.mean(axis=0)
        results[i]['mse'] = np.mean(mse)
        results[i]['r2_scores'] = np.mean(r2_scores)
        results[i]['p_values'] = p_values
    return results


In [24]:
import itertools

def run_significance_alpha_pair_test(training_lengths, results):
    pairs = itertools.permutations(training_lengths, 2)

    significance_alpha = dict()
    for pair in pairs:
        print(pair)
        if pair not in significance_alpha:
            results_1 = results[pair[0]]
            results_2 = results[pair[1]]
            significance_alpha[pair] = perform_significance_test(alpha_1=[result['alpha'] for result in results_1 if 'alpha' in result],
                                                                 alpha_2=[result['alpha'] for result in results_2 if 'alpha' in result])
        return significance_alpha

In [13]:
def get_beta_results(results):

    beta_results = dict()

    for key, item in results.items():
        if key not in beta_results:
            beta_results[key] = dict()
            for i, stock in enumerate(item):
                if 'betas' not in stock:
                    continue
                for factor_beta_column in stock['betas'].index:
                    if factor_beta_column not in beta_results[key]:
                        beta_results[key][factor_beta_column] = dict()
                    beta_results[key][factor_beta_column][i] = stock['betas'][factor_beta_column]
    return beta_results

In [14]:
def print_significance_results(significance_beta, significance_alpha, column_beta='HML'):
    print(f'BETA {column_beta} \n\n')
    for key, item in significance_beta.items():
        print(key)
        print('One-sided')
        print(item[column_beta]['betas']['less'].pvalue)
        print('Two-Sided')
        print(item[column_beta]['betas']['two-sided'].pvalue)
    print('\n\nALPHA')
    for key, item in significance_alpha.items():
        print(key)
        print('Less')
        print(item['alpha']['less'].pvalue)
        print('Two-Sided')
        print(item['alpha']['two-sided'].pvalue)

In [15]:
def plot_p_distribution(training_lengths, results, title_suffix):
    p_results = dict()
    for training_length in training_lengths:
        p_results[training_length] = pd.concat([result['p_values'] for result in results[100]]).dropna()

    for key, p_result in p_results.items():
        for column in p_result.columns:
            values = p_result[column].copy()
            values = values.sort_values()
            title = 'alpha' if column == 'const' else f'beta_{column}'
            plt.suptitle(f'Training Length: {key}, {title}', fontsize=20)
            title += f'_{title_suffix}'
            values.plot(kind='hist', bins=100)
            plt.xlabel('p-value distribution')
            plt.savefig(f'plots/{title}_{key}_{column}.png')
            plt.show()

In [16]:
def run_significance_beta_pair_test(training_lengths, beta_results, names):

    pairs = itertools.permutations(training_lengths, 2)

    significance_beta = dict()
    for pair in pairs:
        if pair not in significance_beta:
            significance_beta[pair] = dict()
        for name in names:
            if name not in significance_beta[pair]:
                betas_1 = list(beta_results[pair[0]][name].values())
                betas_2 = list(beta_results[pair[1]][name].values())
                significance_beta[pair][name] = perform_significance_test(betas_1=betas_1,
                                                                                 betas_2=betas_2)
    return significance_beta

In [17]:
def run_experiment_multiple_training_length(training_lengths, names, regressor=factors, regressor_on='Unnamed: 0'):
    is_factors = regressor_on == 'Unnamed: 0'
    results = dict()
    for training_length in training_lengths:
        if training_length not in results:
            results[training_length] = run_experiment_single_training_length(regressor,
                                                                             regressor_on,
                                                                             training_length=training_length,
                                                                             stocks=stock_data.copy())


    significance_alpha = run_significance_alpha_pair_test(training_lengths, results)
    beta_results = get_beta_results(results)
    significance_beta = run_significance_beta_pair_test(training_lengths, beta_results, names)
    print_significance_results(significance_beta, significance_alpha, column_beta='HML' if is_factors else 'Aero')
    plot_p_distribution(training_lengths, results, 'factors' if is_factors else 'sectors')



In [22]:
training_lengths = np.linspace(100, 250, num=4, dtype=int)
factor_names = list(factors.columns.difference(['Unnamed: 0', 'Mkt-RF', 'RF'])).copy()
sector_names = list(sectors.columns.difference(['Row', 'Market_RF'])).copy()


In [25]:
run_experiment_multiple_training_length(training_lengths, factor_names, regressor=factors, regressor_on='Unnamed: 0')

1781
Running on 0


/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/li

Running on 163


/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/li

Running on 335


...

/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/li

Running on 335


/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/li

(100, 150)


ALPHA
(100, 150)
Less
8.882575244454649e-06
Two-Sided
1.7765150488909298e-05


/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/li

In [20]:
run_experiment_multiple_training_length(training_lengths, sector_names, regressor=sectors, regressor_on='Row')



1781
Running on 0


/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/li

Running on 163


/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/li

Running on 335


...

1781
Running on 0


/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/li

Running on 163


/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/ravinkohli/rosetta-anaconda3/envs/economics/lib/python3.9/site-packages/statsmodels/regression/li

KeyboardInterrupt: 